In [85]:
import pandas as pd

In [100]:
### Variables that you might want to modify:

#local path for California's Google reviews and metadata downloaded from
# https://datarepo.eng.ucsd.edu/mcauley_group/gdrive/googlelocal/#files
reviews_path = '/Users/andres/Documents/Flavor Finder data/California Google Local Reviews.json'
metadata_path = '/Users/andres/Documents/Flavor Finder data/California Local Reviews.json'


#local folder path where you want your clean data to be saved
FFreviews_path = '/Users/andres/Documents/Flavor Finder data/'
FFmetadata_path = '/Users/andres/Documents/Flavor Finder data/'



# number of reviews to read at a time, you might want it smaller if your computer doesn't have lots of RAM
chunk_size = 10000

#min review rating to consider
min_rating = 3

#Lists of columns to be dropped on each database
#Original colums are
    # reviews: ['user_id', 'name', 'time', 'rating', 'text', 'pics', 'resp', 'gmap_id']
    # metadata ['name', 'address', 'gmap_id', 'description', 'latitude', 'longitude',
           #'category', 'avg_rating', 'num_of_reviews', 'price', 'hours', 'MISC',
           #'state', 'relative_results', 'url']
ReviewColumnsToDrop = ['time','pics', 'resp']
MetadataColumnsToDrop = ['price', 'hours', 'MISC','state', 'relative_results', 'url']

#---------------------------------------------------------------------------------------------------------

In [87]:
metadataDF = pd.read_json(metadata_path,lines=True)

In [88]:
#dropping metada rows that do not correspond to restaurants
category_to_check = 'restaurant'
metadataDF = metadataDF.dropna(subset=['category'])
mask = metadataDF['category'].apply(lambda x: any(category_to_check in item for item in x)  )
metadataDF= metadataDF[mask]
metadataDF.drop(columns = MetadataColumnsToDrop, inplace = True)




In [89]:
##Reading and cleaning reviews in chunks

#List of chunks, to be join as one data frame after processing 
chunks = []

# Iterate over the JSON file in chunks
for chunk in pd.read_json(reviews_path, lines=True, chunksize=chunk_size):
    
    #dropping columns that won't be used
    chunk.drop(columns = ReviewColumnsToDrop, inplace = True)
    
    #dropping reviews with rating < min_rating
    mask = chunk.rating >= min_rating
    chunk = chunk[mask]

    #dropping reviews with no text
    chunk = chunk.dropna(subset=['text'])

    #filtering reviews of restaurants
    chunk = chunk[chunk['gmap_id'].isin(metadata_df['gmap_id'])]
    
    #adding the cleaned data to the list of chunks
    chunks.append(chunk)
    

In [90]:
reviewsDF = pd.concat(chunks, ignore_index=True)

In [91]:

metadataDF.to_json(FFreviews_path + 'restMetadata.json', orient='records', lines=True)
reviewsDF.to_json(FFreviews_path + 'restReviews.json', orient='records', lines=True)

In [92]:
#reviewsDF.head()

,user_id,name,rating,text,gmap_id
0,1.078898e+20,jane moon,5.0,One of the best Korean rice cake places in town,0x80c2c778e3b73d33:0xbdc58662a4a97d49
1,1.089912e+20,Song Ro,5.0,Love there korean rice cake.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
2,1.112903e+20,Rafa Robles,5.0,Good very good,0x80c2c778e3b73d33:0xbdc58662a4a97d49
3,1.126404e+20,David Han,4.0,They make Korean traditional food very properly.,0x80c2c778e3b73d33:0xbdc58662a4a97d49
4,1.174403e+20,Anthony Kim,5.0,Short ribs are very delicious.,0x80c2c778e3b73d33:0xbdc58662a4a97d49


In [97]:
#grouped = reviewsDF.groupby('gmap_id').count()
#grouped

,user_id,name,rating,text
gmap_id,,,,
0x100277924bd075cf:0x580587e75a2e5930,109,109,109,109
0x14e01eae3c43bb3b:0x45792563427359ce,292,292,292,292
0x14e030b1ff1b4bf7:0x7b413e7f529d4c35,63,63,63,63
0x14e03579d022efd1:0x1142b59e50968d8,19,19,19,19
0x14e035bf1d6e56af:0x4b5a1833b3b81a05,145,145,145,145
...,...,...,...,...
0x89c2f60a6f4f1205:0xa670128f9c033c7b,4,4,4,4
0x89c3c775b83ea805:0x348cfd6154f8eb19,13,13,13,13
0x89d312304c2eba19:0x1f0feb9ba12e4a1e,111,111,111,111


In [99]:
#grouped.describe()

,user_id,name,rating,text
count,78685.000000,78685.000000,78685.000000,78685.000000
mean,163.970312,163.970312,163.970312,163.970312
std,232.017899,232.017899,232.017899,232.017899
min,1.000000,1.000000,1.000000,1.000000
25%,38.000000,38.000000,38.000000,38.000000
50%,97.000000,97.000000,97.000000,97.000000
75%,207.000000,207.000000,207.000000,207.000000
max,9743.000000,9743.000000,9743.000000,9743.000000
